In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import warnings
import sklearn
import random
import xgboost as xgb
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import roc_curve, auc, f1_score, accuracy_score
from sklearn.metrics import precision_recall_curve, average_precision_score
from sksurv.metrics import cumulative_dynamic_auc, concordance_index_censored
import ast

warnings.filterwarnings("ignore")


In [ ]:
def train_val_split(deriv_data, shuffle=True, random_state=42):
    # Divide patients to train / validation / groups
    
    random.seed(random_state)
    # Divide patients to train / validation / groups
    
    patient_list = deriv_data['henkilotunnus'].unique()
    
    if shuffle == True:
        random.shuffle(patient_list)
    
    # Calculate the number of items in each sublist
    total_items = len(patient_list)
    train_size = int(total_items * 0.85)
    val_size = total_items - train_size  # To ensure all items are included

    # Divide the list into sublists
    train_list = patient_list[:train_size]
    val_list = patient_list[train_size:]
    
    train_data = deriv_data[deriv_data['henkilotunnus'].isin(train_list)].reset_index(drop=True)
    val_data = deriv_data[deriv_data['henkilotunnus'].isin(val_list)].reset_index(drop=True)

    return train_data, val_data

In [ ]:
my_path = '~/mounts/research/husdatalake/disease/scripts/Preleukemia/oona_git'

In [ ]:
disease = 'MDS'

### Read data, features and hyperparameters

In [ ]:
deriv_data = pd.read_csv(my_path + '/data/modelling/' + disease + '_derivation_data.csv')

In [ ]:
test_data = pd.read_csv(my_path + '/data/modelling/' + disease + '_test_data.csv')

In [ ]:
features = pd.read_csv(my_path + '/optimization/feature_selection/' + disease + '_features.csv')['features'].to_list()

In [ ]:
deriv_data = deriv_data[['henkilotunnus','time_to_dg','disease'] + features]
test_data = test_data[['henkilotunnus','time_to_dg','disease'] + features]

In [ ]:
# Remove columns starting 'e_retic'
deriv_data = deriv_data.loc[:, ~deriv_data.columns.str.startswith('e_retic')]
test_data = test_data.loc[:, ~test_data.columns.str.startswith('e_retic')]

In [ ]:
print(f'Using {len(deriv_data.columns)} features in the final model')

In [ ]:
# Read hyperparameters
hyperparams = pd.read_csv(my_path + '/optimization/hyperparams/' + disease + '_hyperparameter_results_cv.csv', index_col=0)
max_idx = hyperparams['AUCPR_mean'].idxmax()
params = ast.literal_eval(hyperparams['params'].loc[max_idx])
params

In [ ]:
nrounds = 1000
early_stop = 10

### Define binary classification threshold with 10-fold cross-validation

In [ ]:
cv_result_df = pd.DataFrame(index=range(1), columns=['c_index_mean', 'c_index_std', 'AUC_mean', 'AUC_std', 'AUCPR_mean', 'AUCPR_std'])

In [ ]:
cv = 10

In [ ]:
c_indices = []
AUCs = []
AUCPRs = []
youden_indices = []

In [ ]:
print(f'\nFINDING BINARY CLASSIFICATION THRESHOLD - {cv}-FOLD CROSS VALIDATION')

In [ ]:
for i in range(cv):

    print('\n\tCV loop no: ', i+1)
    
    train_data, validation_data = train_val_split(deriv_data, shuffle=True, random_state=None)
    
    # Check the class ratios
    pos_ratio_train = 100 * train_data['disease'].value_counts()[1] / train_data['disease'].value_counts()[0]
    pos_ratio_val = 100 * validation_data['disease'].value_counts()[1] / validation_data['disease'].value_counts()[0]
    pos_ratio_test = 100 * test_data['disease'].value_counts()[1] / test_data['disease'].value_counts()[0]
    print(f'\n{pos_ratio_train} % of the datapoints in the training set had disease = 1')
    print(f'{pos_ratio_val} % of the datapoints in the validation set had disease = 1')

    # Sanity check - is any of test indices in validation or training sets
    print('\nSanity check: Is there any validaion data in train set')
    train_ht = list(train_data['henkilotunnus'].unique())
    validation_ht = list(validation_data['henkilotunnus'].unique())
    test_ht = list(test_data['henkilotunnus'].unique())
    val_in_train = np.intersect1d(validation_ht, train_ht).size > 0
    print(val_in_train)

    # Separate features and target variables
    x_train = train_data.drop(columns=['henkilotunnus', 'disease', 'time_to_dg'])
    y_train = train_data['time_to_dg']

    x_val = validation_data.drop(columns=['henkilotunnus', 'disease', 'time_to_dg'])
    y_val = validation_data['time_to_dg']

    # Create DMatrix for XGBoost
    dtrain = xgb.DMatrix(x_train, label=y_train)
    dval = xgb.DMatrix(x_val, label=y_val)
    
    # Use validation set to watch performance
    watchlist = [(dtrain,'train'), (dval,'eval')]

    # Store validation results
    evals_results = {}

    # Train the model
    xgb_model = xgb.train(params, dtrain, num_boost_round=nrounds, early_stopping_rounds=early_stop, evals=watchlist, evals_result=evals_results, verbose_eval=50)

    # Predict risk scores
    risk_scores_train = xgb_model.predict(dtrain)
    risk_scores_val = xgb_model.predict(dval)

    # Add risk scores to the dataframe
    train_data['risk_score'] = risk_scores_train
    validation_data['risk_score'] = risk_scores_val
    
    # Calculate C-index for validation set
    # Negative times to positive for getting c-index
    validation_data['time_to_dg'] = validation_data['time_to_dg'].apply(lambda x: -x if x < 0 else x)
    c_index = concordance_index_censored(event_indicator=validation_data['disease'].replace({0 : False, 1 : True}), event_time=validation_data['time_to_dg'], estimate=validation_data['risk_score'])[0]
    
    # AUC-ROC
    fpr, tpr, thresholds = roc_curve(validation_data['disease'], validation_data['risk_score'])
    roc_auc = auc(fpr, tpr)

    # Compute Youden Index for each threshold
    youden_index = tpr - fpr
    optimal_threshold_index = np.argmax(youden_index)
    optimal_threshold = thresholds[optimal_threshold_index]
    optimal_fpr = fpr[optimal_threshold_index]
    optimal_tpr = tpr[optimal_threshold_index]
    
    youden_indices.append(optimal_threshold)

    # Plotting the ROC curve
    fig = plt.figure(figsize=(6,6))
    plt.plot(fpr, tpr, lw=3, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.scatter(optimal_fpr, optimal_tpr, color='r', zorder=5, label='Youden Index', marker='o',s=100)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', alpha=0.3)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate', fontsize=15)
    plt.ylabel('True Positive Rate', fontsize=15)
    plt.title(f'Validation data', fontsize=15)
    plt.xticks(fontsize=15, rotation=0)
    plt.yticks(fontsize=15, rotation=0)
    plt.legend(loc="lower right")
    sns.despine(fig=fig, ax=None, top=True, right=True, left=False, bottom=False, offset=None, trim=False)
    plt.show()

    # Calculate precision and recall
    precision, recall, pr_thresholds = precision_recall_curve(validation_data['disease'], validation_data['risk_score'])
    average_precision = average_precision_score(validation_data['disease'], validation_data['risk_score'])
    
    print(f"Youden index for for validation data: {optimal_threshold}")
    
    c_indices.append(c_index)
    AUCs.append(roc_auc)
    AUCPRs.append(average_precision)

In [ ]:
cv_result_df.loc[0]['c_index_mean'] = np.mean(c_indices)
cv_result_df.loc[0]['AUC_mean'] = np.mean(AUCs)
cv_result_df.loc[0]['AUCPR_mean'] = np.mean(AUCPRs)

cv_result_df.loc[0]['c_index_std'] = np.std(c_indices)
cv_result_df.loc[0]['AUC_std'] = np.std(AUCs)
cv_result_df.loc[0]['AUCPR_std'] = np.std(AUCPRs)

In [ ]:
cv_result_df

### Use average youden index on validation data across 10 cv loops as binary threshold 

In [ ]:
binary_threshold = np.mean(youden_indices)

In [ ]:
binary_threshold

In [ ]:
# Define pre-trained threshold
binary_threshold = 0.740484

### Train final model

In [ ]:
train_data, validation_data = train_val_split(deriv_data, shuffle=True, random_state=42)

In [ ]:
# Separate features and target variables
x_train = train_data.drop(columns=['henkilotunnus', 'disease', 'time_to_dg'])
y_train = train_data['time_to_dg']

x_val = validation_data.drop(columns=['henkilotunnus', 'disease', 'time_to_dg'])
y_val = validation_data['time_to_dg']

x_test = test_data.drop(columns=['henkilotunnus', 'disease', 'time_to_dg'])
y_test = test_data['time_to_dg']

In [ ]:
# Save x_train for getting SHAP values
x_train.to_csv('results/final_model/SHAP/' + disease + '_x_train.csv', index=False)

In [ ]:
len(deriv_data) + len(test_data)

In [ ]:
len(x_train) + len(x_val) + len(x_test)

In [ ]:
# Create DMatrix for XGBoost
dtrain = xgb.DMatrix(x_train, label=y_train)
dval = xgb.DMatrix(x_val, label=y_val)
dtest = xgb.DMatrix(x_test, label=y_test)

In [ ]:
# Use validation set to watch performance
watchlist = [(dtrain,'train'), (dval,'eval')]

# Store validation results
evals_results = {}

# Train the model
print(f'\nTraining the model with parameters: ')
print(params)

xgb_model = xgb.train(params, dtrain, num_boost_round=nrounds, early_stopping_rounds=early_stop, evals=watchlist, evals_result=evals_results, verbose_eval=50)

In [ ]:
# Training and validation losses
tr_loss = list(evals_results['train'].values())[0]
val_loss = list(evals_results['eval'].values())[0]
plt.plot(range(len(tr_loss)), tr_loss, label='Training loss')
plt.plot(range(len(tr_loss)), val_loss, label='Validation loss')
plt.legend()
plt.show()

In [ ]:
xgb_model.save_model('results/final_model/' + disease + '_final_model.json')

In [ ]:
# Predict risk scores
risk_scores_train = xgb_model.predict(dtrain)
risk_scores_val = xgb_model.predict(dval)
risk_scores_test = xgb_model.predict(dtest)

# Add risk scores to the dataframe
train_data['risk_score'] = risk_scores_train
validation_data['risk_score'] = risk_scores_val
test_data['risk_score'] = risk_scores_test

### Predictions

In [ ]:
test_data['time_to_dg'] = test_data['time_to_dg'].apply(lambda x: -x if x < 0 else x)

In [ ]:
# Convert risk scores to binary predictions using the optimal threshold
predicted_labels = (test_data['risk_score'] >= binary_threshold).astype(int)
test_data['predicted_disease'] = predicted_labels

In [ ]:
# Save dataset with final model predictions for plotting
test_data.to_csv(my_path + '/results/final_model/' + disease + '_test_data_with_final_model_predictions.csv', index=False)